In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
tfd = tfp.distributions
tfl = tf.linalg

### Generalized Linear Models

### TensorFlow State Space Models

In [2]:
ndims = 2
step_std = 1.0
noise_std = 5.0

In [3]:
model = tfd.LinearGaussianStateSpaceModel(
  num_timesteps=100,
  transition_matrix=tfl.LinearOperatorIdentity(ndims),
  transition_noise=tfd.MultivariateNormalDiag(
   scale_diag=step_std**2 * tf.ones([ndims])),
  observation_matrix=tfl.LinearOperatorIdentity(ndims),
  observation_noise=tfd.MultivariateNormalDiag(
   scale_diag=noise_std**2 * tf.ones([ndims])),
  initial_state_prior=tfd.MultivariateNormalDiag(
   scale_diag=tf.ones([ndims]))
)


In [4]:
x = model.sample(5) # Sample from the prior on sequences of observations.
lp = model.log_prob(x) # Marginal likelihood of a (batch of) observations.

Instructions for updating:
Previously, `initial_step` would return the value passed to the constructor, even if that value was not `Tensor`-like. This behavior is now deprecated. In the future, it will always return a `Tensor`-like object.


In [5]:
# Run Kalman filter over provided input sequence
ll, filt_mean, filt_cov, pred_mean, pred_covs, obs_mean, obs_cov = model.forward_filter(x) 

In [32]:
# Compute the filtered posterior on latent states given observations,
# and extract the mean and covariance for the current (final) timestep.
current_location_posterior = tfd.MultivariateNormalFullCovariance(
              loc=filt_mean,
              covariance_matrix=filt_cov)

In [38]:
# Run a smoothing recursion to extract posterior marginals for locations
# at previous timesteps.
post_mean, post_cov = model.posterior_marginals(x)

In [40]:
initial_location_posterior = tfd.MultivariateNormalFullCovariance(
              loc=post_mean,
               covariance_matrix=post_cov)


In [41]:
initial_location_posterior

<tfp.distributions.MultivariateNormalFullCovariance 'MultivariateNormalFullCovariance' batch_shape=[5, 100] event_shape=[2] dtype=float32>

In [42]:
post_mean

<tf.Tensor: shape=(5, 100, 2), dtype=float32, numpy=
array([[[-8.63413960e-02, -2.67389387e-01],
        [-2.14592665e-01, -4.94236410e-01],
        [-3.89290482e-01, -6.96058035e-01],
        [-5.67954779e-01, -9.18911099e-01],
        [-7.66269028e-01, -1.09316504e+00],
        [-9.97505605e-01, -1.33102584e+00],
        [-1.20121765e+00, -1.55314660e+00],
        [-1.49744737e+00, -1.71365964e+00],
        [-1.75122845e+00, -1.90128756e+00],
        [-1.96213937e+00, -2.07654428e+00],
        [-2.12413359e+00, -2.25211477e+00],
        [-2.39427733e+00, -2.34743285e+00],
        [-2.65756607e+00, -2.41786766e+00],
        [-2.94305182e+00, -2.44271183e+00],
        [-3.19096994e+00, -2.43356681e+00],
        [-3.42702436e+00, -2.48145437e+00],
        [-3.64879751e+00, -2.47966290e+00],
        [-3.91382241e+00, -2.38325882e+00],
        [-4.15828085e+00, -2.24627924e+00],
        [-4.40551710e+00, -2.07646942e+00],
        [-4.66704321e+00, -1.97137332e+00],
        [-4.96708632e+0

In [ ]:
tfp.mcmc.NoUTurnSampler(
    target_log_prob_fn, step_size, max_tree_depth=10, max_energy_diff=1000.0,
    unrolled_leapfrog_steps=1, parallel_iterations=10, seed=None, name=None
)
